Analysis

In [648]:
import itertools
import numpy as np
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x

In [649]:
import pandas as pd

In [868]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True)
cl6=pd.read_json('cl6.json').reset_index(drop=True)
cl6['sltn']=cl6['sltn'].apply(jsonKeys2int)
cl6=cl6.sort_values('n').reset_index(drop=True)

DD analysis

In [869]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [870]:
def extract(l,x):
    for i in range(5):
        try:
            l.remove(x)
        except:
            break
    return l

def remove_list(l,pr):
    for x in pr:
        try:
            l=extract(l,x)
        except:
            pass
    return l

def remove(l,pr):
    if pr:
        pr=np.concatenate(pr).ravel()
    l=remove_list(l,pr)
    return l

def get_massless_fermions(l,ν,m=0):
    s=-(ν+m)
    l=extract(l,ν)
    if m:
        l=extract(l,m)
    #Dirac masses
    for ss in [s,-s]:
        pr=[ p for  p in get_permuations( l,n=2  ) if np.append( p,ss).sum()==0 ]
        l=remove(l,pr)

    #Majorana masses
    for ss in [s,-s]:
        pr=[ll for ll in l if 2*ll+ss==0]
        l=remove_list(l,pr)
    return list(np.unique(l))

def count_massless_fermions(l,ν,m=0):
    return len( get_massless_fermions(l,ν,m) )

In [871]:
l=[2, -3, -10, 13, 13, -15]
l=[3, -4, -21, 25, 25, -28]
l=[8, 9, 11, -13, -20, -20, 25]
l=[2, 2, -3, 12, -14, -18, 19]
l=[5, 5, 7, -8, -12, -12, 15]
#l=[6, 8, 11, -14, -14, -18, 21]
#l=[3, 3, -8, -15, 20, 20, -23]
#l=[1, -3, 6, -7, -14, 21, 21, -25]
#l=[1, 1, -4, -5, 9, 9, 9, -10, -10]
l=[3, 3, -6, -9, 10, 12, -15, -15, 17]
l=[1, 3, -7, -9, 13, 24, -27, -27, 29]
#XD
l=[2, -5, -5, -5, 7, 8, 8, -10]

In [872]:
get_massless_fermions([2, -3, -3, -5, 9, -13, 15, 16, -18],ν=-3)

[9]

In [873]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=0)==2

True

In [874]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=-5)==1

True

In [875]:
count_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13)==0

True

In [876]:
count_massless_fermions( [5, 13, 13, -16, -18, -23, 26],ν=13)==3

True

In [877]:
get_massless_fermions( [2, -4, 7, 9, -18, -18, 22],ν=-18 )

[2, 7]

In [878]:
def massfer(row,label='DarkDirac',nmax=10):
    # 
    if label=='DarkDirac' or label=='DarkMajor':
        if label=='DarkDirac':
            nuR='DD'
        else:
            nuR='DM'
        if row.get(label)>0:
            cdd=[]
            for ν in row.get(nuR):
                l=row.get('solution').copy()
                cdd.append( count_massless_fermions(l , ν)  )
            return min(cdd)
        else:
            return nmax
    if label=='XDirac' or label=='XMajor':
        if label=='XDirac':
            nuR='XD'
        else:
            nuR='XM'
        if row.get(label)>0:
            cddmax=100
            for m in row['sltn'].get(3):
                cdp=[]
                for ν in row.get(nuR):
                    l=row.get('solution').copy()
                    l=[ll for ll in l if ll!=m]
                    cdp.append( count_massless_fermions(l , ν,m)  )
                if min(cdp)<cddmax:
                    cdd=cdp
            return min(cdd)
        else:
            return nmax    

In [879]:
row=cl.loc[4].to_dict()

In [880]:
massfer(row,label='XMajor')

10

In [882]:
cl['DD_massless']=cl.apply(lambda row: massfer(row,label='DarkDirac'),axis='columns')
cl['DM_massless']=cl.apply(lambda row: massfer(row,label='DarkMajor'),axis='columns')
cl['XD_massless']=cl.apply(lambda row: massfer(row,label='XDirac'),axis='columns')
cl['XM_massless']=cl.apply(lambda row: massfer(row,label='XMajor'),axis='columns')

In [ ]:
#TODO 
cl6['DD_massless']=cl6.apply(lambda row: massfer(row,label='DarkDirac'),axis='columns')
cl6['DM_massless']=cl6.apply(lambda row: massfer(row,label='DarkMajor'),axis='columns')
cl6['XD_massless']=cl6.apply(lambda row: massfer(row,label='XDirac'),axis='columns')
cl6['XM_massless']=cl6.apply(lambda row: massfer(row,label='XMajor'),axis='columns')

In [854]:
cl[cl.apply(lambda row: massfer(row,label='DarkDirac'),axis='columns')<10][:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,total,XD_massless
0,"[-5, 0]","[1, -1]",6,"[2, -3, -10, 13, 13, -15]",2,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",2,0,0,0,{2: [13]},[13],0,0,0,0,56,10
1,"[4, 7]","[21, 18]",6,"[3, -4, -21, 25, 25, -28]",7749,"[{'D→nu_R+fi+fj': [-28, 3, 25]}, {'D→nu_R+fi+fj': [-21, -4, 25]}]",2,0,0,0,{2: [25]},[25],0,0,0,0,106,10
2,"[-4, 0]","[-1, 1]",6,"[3, -5, -12, 17, 17, -20]",1,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",2,0,0,0,{2: [17]},[17],0,0,0,0,74,10


In [855]:
cl['total']=cl['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )

In [866]:
cl[ (cl['n']>0) &  (cl['DarkMajor']>0) ].sort_values(['n','DM_massless','total'])

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,total,XD_massless,DM_massless,XM_massless
81,"[1, 0, -1]","[-4, -6]",7,"[1, 1, -3, -4, 6, 6, -7]",3,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1,1,0,0,"{2: [1, 6]}","[1, 6]",[6],0,0,1,28,1,1,10
57,"[-1, -4, -3]","[2, 0]",7,"[1, 3, -4, 5, -6, -6, 7]",3,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,1,0,0,{2: [-6]},[-6],[-6],0,0,2,32,2,2,10
10,"[-5, -9, 7]","[5, 15]",7,"[2, -4, 7, 9, -18, -18, 22]",625,"[{'D→nu_R+2fi': [-18, 9]}, {'D→nu_R+fi+fj': [-18, -4, 22]}]",1,1,0,0,{2: [-18]},[-18],[-18],0,0,2,80,2,2,10
63,"[6, 11, 9]","[6, 3]",7,"[4, -5, 7, 11, -22, -22, 27]",63,"[{'D→nu_R+2fi': [-22, 11]}, {'D→nu_R+fi+fj': [-22, -5, 27]}]",1,1,0,0,{2: [-22]},[-22],[-22],0,0,2,98,2,2,10
73,"[-4, -6, -4]","[9, 12]",7,"[8, 9, 10, -14, -18, -18, 23]",232,"[{'D→nu_R+2fi': [-18, 9]}, {'D→nu_R+fi+fj': [-18, 8, 10]}]",1,1,0,0,{2: [-18]},[-18],[-18],0,0,2,100,2,2,10
27,"[10, 0, 15]","[-5, -10]",7,"[9, 11, 13, -17, -22, -22, 28]",1250,"[{'D→nu_R+2fi': [-22, 11]}, {'D→nu_R+fi+fj': [-22, 9, 13]}]",1,1,0,0,{2: [-22]},[-22],[-22],0,0,2,122,2,2,10
26,"[-2, -6, 12]","[4, -5]",7,"[3, 6, -12, -12, -12, 13, 14]",120,"[{'D→nu_R+2fi': [-12, 6]}]",0,1,0,0,{3: [-12]},0,[-12],0,0,10,72,3,3,10
37,"[-5, 5, -4]","[-6, -3]",7,"[6, 7, 9, -12, -12, -18, 20]",105,"[{'D→nu_R+2fi': [-12, 6]}]",0,1,0,0,{2: [-12]},0,[-12],0,0,10,84,3,3,10
33,"[-1, 0, 3]","[2, -2]",7,"[4, 5, 7, -9, -10, -10, 13]",1,"[{'D→nu_R+2fi': [-10, 5]}]",0,1,0,0,{2: [-10]},0,[-10],0,0,10,58,4,4,10
32,"[-9, 3, -1]","[12, 18]",7,"[3, -4, -6, -6, 15, 21, -23]",756,"[{'D→nu_R+2fi': [-6, 3]}]",0,1,0,0,{2: [-6]},0,[-6],0,0,10,78,4,4,10


In [734]:
cl[ (cl['n']==9) &  (cl['DarkDirac']>0) ].sort_values(['DD_massless','total'])

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,total
3735,"[4, 8, 6, 3]","[-2, 3, 0]",9,"[1, 1, 2, 2, 3, -5, -6, -6, 8]",4,"[{'D→nu_R+fi+fj': [-5, 2, 3]}, {'D→nu_R+2fi': [-6, 3]}]",1,1,0,0,"{2: [1, 2, -6]}",[2],[-6],0,0,0,34
3388,"[1, 0, -5, 9]","[-3, -1, -2]",9,"[1, -2, 3, 4, 6, -7, -7, -7, 9]",2,"[{'D→nu_R+fi+fj': [-7, -2, 9]}, {'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+fi+fj': [-7, 3, 4]}]",3,0,0,0,{3: [-7]},[-7],0,0,0,0,46
3334,"[4, 10, 14, 3]","[16, 2, 10]",9,"[1, -3, -3, -4, -5, 8, 9, 9, -12]",1272,"[{'D→nu_R+fi+fj': [-5, -3, 8]}, {'D→nu_R+fi+fj': [-5, -4, 9]}]",2,0,0,0,"{2: [-3, 9]}","[-3, 9]",0,0,0,0,54
3412,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1,"{2: [1, -10], 3: [9]}",[9],0,0,[1],0,58
3672,"[1, 12, 0, -3]","[1, 0, -1]",9,"[1, 1, -3, -8, 11, 11, 11, -12, -12]",14,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'D→nu_R+fi+fj': [-8, -3, 11]}]",2,0,0,0,"{2: [1, -12], 3: [11]}",[11],0,0,0,0,70
3224,"[4, -2, 0, 11]","[10, 6, -4]",9,"[4, -5, -5, -6, -6, 11, 11, 11, -15]",392,"[{'D→nu_R+fi+fj': [-6, -5, 11]}, {'D→nu_R+fi+fj': [-15, 4, 11]}]",2,0,0,0,"{2: [-5, -6], 3: [11]}",[11],0,0,0,0,74
2008,"[4, -3, -4, -1]","[-4, 0, 16]",9,"[1, -3, 4, 9, 12, -13, -13, -13, 16]",48,"[{'D→nu_R+fi+fj': [-13, -3, 16]}, {'D→nu_R+fi+fj': [-13, 1, 12]}, {'D→nu_R+fi+fj': [-13, 4, 9]}]",3,0,0,0,{3: [-13]},[-13],0,0,0,0,84
3088,"[-5, 0, -10, -7]","[10, 16, 5]",9,"[2, -5, -5, -7, -10, 15, 15, 16, -21]",605,"[{'D→nu_R+fi+fj': [-10, -5, 15]}]",1,0,0,0,"{2: [-5, 15]}","[-5, 15]",0,0,0,0,96
3408,"[2, -1, 0, -10]","[2, 0, -2]",9,"[1, 2, -5, -10, 15, 15, 15, -16, -17]",28,"[{'D→nu_R+fi+fj': [-17, 2, 15]}, {'D→nu_R+fi+fj': [-16, 1, 15]}, {'D→nu_R+fi+fj': [-10, -5, 15]}]",3,0,0,0,{3: [15]},[15],0,0,0,0,96
3402,"[10, 6, -9, 10]","[10, 18, 14]",9,"[2, -4, 5, 10, 13, -15, -15, -15, 19]",1336,"[{'D→nu_R+fi+fj': [-15, -4, 19]}, {'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-15, 5, 10]}]",3,0,0,0,{3: [-15]},[-15],0,0,0,0,98


def repeated(f):
    r={}
    r[1]=f
    for ri in range(2,f.size+2):
        r[ri]=r[ri-1][pd.Series(r[ri-1]).duplicated()]
        if ri>2:
            for a in np.unique(r[ri]):
                r[ri-1]=r[ri-1][ r[ri-1]!=a ]
                if len(r[ri-1])>0:
                    r[ri-1][ r[ri-1]!=a ]
        if len(r[ri])<=1:
            r[ri]=r[ri]
            r.pop(1)
            break
    return dict( (k,list(v)) for k,v in r.items() if len(v)!=0 )

In [678]:
cl[2799:2800]['solution'].apply(lambda s: repeated(np.asarray(s)))

2799    {2: [-23], 3: [2]}
Name: solution, dtype: object

In [679]:
r={2: [-23], 3: [2]} 

In [683]:
dict( ((k,v) for k,v in r.items() if k!=3) )

{2: [-23]}

In [645]:
def get_nuR_i(row,i,model,label):
    print( row[model]>0,'**',  row['sltn'].get(i))
    if row[model]>0 and row['sltn'].get(i) and row['nu_R'] and len( [ d.get(label) for d in row['nu_R'] if d.get(label)] )>0:
        M=[m  for m in  row['nu_R'] if m.get(label)]
        nuR=[n for n in row['sltn'][i] for m in M  if n in m.get(label)  ]
        #print(nuR)
        if nuR:
            return list(np.unique(nuR))
    else:
        return []

In [647]:
kk.apply(lambda row: get_nuR_i(row,3,'DarkDirac','D→nu_R+fi+fj'),axis='columns')

True ** None


1515    []
dtype: object

In [601]:
kk.sltn.loc[1542]['4']

[2]

In [640]:
cl[1537:1538]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
1537,"[6, -1, 7, 1]","[12, 0, 9]",9,"[2, -3, -3, -5, 9, -13, 15, 16, -18]",432,"[{'D→nu_R+fi+fj': [-13, -3, 16]}]",1,0,0,0,{2: [-3]},[-3],0,0,0


In [471]:
cl[:3]

,DarkDirac,DarkMajor,XDirac,XMajor,gcd,k,l,n,nu_R,sltn,solution,DD,DM,XD,XM
0,2,0,0,0,2,"[-5, 0]","[1, -1]",6,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",{'2': [13]},"[2, -3, -10, 13, 13, -15]",[13],0,0,0
1,2,0,0,0,7749,"[4, 7]","[21, 18]",6,"[{'D→nu_R+fi+fj': [-28, 3, 25]}, {'D→nu_R+fi+fj': [-21, -4, 25]}]",{'2': [25]},"[3, -4, -21, 25, 25, -28]",[25],0,0,0
2,2,0,0,0,1,"[-4, 0]","[-1, 1]",6,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",{'2': [17]},"[3, -5, -12, 17, 17, -20]",[17],0,0,0


### Tree level neutrino Dirac masses

In [416]:
l=get_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13,m=0)
[ ll for ll in l if ll+ν-s==0]

[]

In [417]:
[ ll for ll in l if ll-ν+s==0]

[]